<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
# Import some necessary modules
%matplotlib inline
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import pandas as pd
import time
import scipy as sp
import pickle
%load_ext autoreload
%autoreload 2

In [3]:
# Import MuSiCal
import musical

# Codes -- Overview

In [4]:
# Read in result after de novo signature discovery. 
with open('./data/validation_internal/Breast-AdenoCA.96.normalizeX_True.nmf.model.pkl', 'rb') as f:
    model_old = pickle.load(f)
# Read in the original data matrix X as pd.DataFrame. This is mostly used for the sample names.
# !!!!!!!!!!!!!!!!!! Make sure that this X matrix precisely matches model_old.X !!!!!!!!!!!!!!!!!!
X = pd.read_csv('./data/validation_internal/Breast-AdenoCA.96.X.csv', index_col=0)
X.head()

,Breast-AdenoCA::SP117956,Breast-AdenoCA::SP117975,Breast-AdenoCA::SP2149,Breast-AdenoCA::SP117113,Breast-AdenoCA::SP117245,Breast-AdenoCA::SP117402,Breast-AdenoCA::SP117136,Breast-AdenoCA::SP117378,Breast-AdenoCA::SP116947,Breast-AdenoCA::SP117369,...,Breast-AdenoCA::SP2731,Breast-AdenoCA::SP6223,Breast-AdenoCA::SP7171,Breast-AdenoCA::SP10563,Breast-AdenoCA::SP8987,Breast-AdenoCA::SP5980,Breast-AdenoCA::SP12186,Breast-AdenoCA::SP10944,Breast-AdenoCA::SP2766,Breast-AdenoCA::SP6673
Type,,,,,,,,,,,,,,,,,,,,,
A[C>A]A,35,36,217,50,35,43,30,22,52,92,...,47,198,71,56,188,193,70,144,162,316
A[C>A]C,29,36,181,20,32,43,27,9,44,48,...,30,158,44,33,169,160,54,111,116,393
A[C>A]G,5,2,36,5,3,7,9,6,8,9,...,7,24,8,3,21,24,9,19,25,48
A[C>A]T,27,22,231,24,24,38,10,12,27,40,...,23,159,47,22,161,184,45,94,139,310
C[C>A]A,25,37,190,33,23,35,15,16,23,86,...,27,134,47,41,166,157,65,90,144,343


In [5]:
# I made some updates on the de novo discovery part of DenovoSig to use pd.DataFrame more, so that
# sample names and signature names are kept. 
# To accomodate that update, the following lines are required to modify old DenovoSig objects. 
# !!!!!!!!!!!!!!!!!! If de novo discovery is performed with new code, no need to do these lines !!!!!!!!!!!!!!!!!!
import copy
model = copy.deepcopy(model_old)
model.X_df = X.astype(float)
model.samples = model.X_df.columns.values
model.features = model.X_df.index.values
model.W_df = pd.DataFrame(model.W, columns=['Sig' + str(i) for i in range(1, model.n_components + 1)],
                          index=model.features)
model.signatures = model.W_df.columns.values
model.H_df = pd.DataFrame(model.H, columns=model.samples, index=model.signatures)

In [6]:
# Read a catalog
# !!!!!!!!!!!!!!!!!! Use an appropriate catalog instead !!!!!!!!!!!!!!!!!!
W_catalog = musical.load_catalog().W

## 2d grid search 

In [7]:
# Define a log-spaced grid. 
# !!!!!!!!!!!!!!!!!! Not sure if this is a good grid. Just as an example !!!!!!!!!!!!!!!!!!
thresh_match_grid = np.geomspace(0.0001, 5, 10)
thresh_refit_grid = np.geomspace(0.0001, 5, 10)
print(thresh_match_grid)

[1.00000000e-04 3.32742119e-04 1.10717318e-03 3.68403150e-03
 1.22583245e-02 4.07886087e-02 1.35720881e-01 4.51600535e-01
 1.50266519e+00 5.00000000e+00]


In [ ]:
# Perform assignment
# Now there is only one thresh for matching and one thresh for refitting. 
# "thresh1" and "thresh2" are now hidden and not accessible to users. 
model.assign_grid(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match_grid=thresh_match_grid,
    thresh_refit_grid=thresh_refit_grid,
    thresh_new_sig=0.0, # Perhaps set this to 0 to suppress calling new signatures? We only care about sig assignment now anyway.
    indices_associated_sigs=None
)

In [ ]:
# Perform validation
model.validate_grid(
    validate_n_replicates=1, # This is the number of repetitions for each grid point. Previously this was 3. 
    W_cos_dist_thresh=0.02 # We consider grid points with W distance <= min + 0.02 as good. 
)
# Note that now the distance between W_data and W_simul is the MEAN cos distance between the signatures. 
# So no need to muliply 0.02 by the number of signatures. 

## Separate 1d grid searches for matching and refitting 

In [ ]:
# First perform 1d grid search for matching. 
# Refitting threshold will be set to a small value. 
model.assign_grid(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match_grid=np.geomspace(0.0001, 5, 10),
    thresh_refit_grid=np.array([0.0001]),
    thresh_new_sig=0.0, 
    indices_associated_sigs=None
)
model.validate_grid(
    validate_n_replicates=1, 
    W_cos_dist_thresh=0.02
)

In [ ]:
# Look at the best matching parameter
best_thresh_match = model.thresh_match

In [ ]:
# Then perform 1d grid search for refitting.
# Matching parameter will be set to the best one above.
# !!!!!!!!!!!!!!!!!!!! Note that if you run assign_grid() and validate_grid() again, the previous result will be lost
# !!!!!!!!!!!!!!!!!!!! So we should save the first grid search result, and then do the second grid search. 
# !!!!!!!!!!!!!!!!!!!! Or use copy.deepcopy() to copy the model to model2, and then do the second grid search. 
model.assign_grid(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match_grid=np.array([best_thresh_match]),
    thresh_refit_grid=np.geomspace(0.0001, 5, 10),
    thresh_new_sig=0.0, 
    indices_associated_sigs=None
)
model.validate_grid(
    validate_n_replicates=1, 
    W_cos_dist_thresh=0.02
)

## Validate a single signature assignment 

In [ ]:
# We can perform a single assignment using model.assign()
# For example:
model.assign(
    W_catalog, 
    method_assign='likelihood_bidirectional',
    thresh_match=0.01, 
    thresh_refit=0.02, 
    thresh_new_sig=0.0, 
    indices_associated_sigs=None
)

In [ ]:
# We can then perform a single validation for this assignment
model.validate(
    W_s=None,
    H_s=None,
    validate_n_replicates=1
)
# Setting W_s and H_s to None will allow it to use the existing W_s and H_s calculated by model.assign()

## Validate external signature assignments 

In [ ]:
# If we just want to validate an external signature assignment result, also use model.validate().
# For example, if W_s_PCAWG and H_s_PCAWG are PCAWG assignments:
model.validate(
    W_s=W_s_PCAWG,
    H_s=H_s_PCAWG,
    validate_n_replicates=1
)
# !!!!!!!!!!!!! Note that W_s_PCAWG and H_s_PCAWG must be pd.DataFrame !!!!!!!!!!!!!
# The sample names in H_s_PCAWG must match with sample names in model.X_df

# Example

Here we do some mock runs to demonstrate the output. 

In order to save time, we have chosen to use a nmf model. And we further simplify it by running less iterations. 

In [8]:
model.max_iter = 10000
model.min_iter = 2000
model.tol = 1e-7
model.n_replicates = 1
model.ncpu = 1

Let's do a 1d search for matching first. 

In [12]:
model.assign_grid(W_catalog, 
                  thresh_match_grid=np.array([0.001, 0.01, 0.1, 1, 5]), 
                  thresh_refit_grid=np.array([0.0001]), 
                  thresh_new_sig=0.0)

/Users/hujin/GitHub/MuSiCal/musical/denovo.py:1056: UserWarning: self.assign_grid has been previously called. This call will replace the previous results.
  UserWarning)


In [15]:
model.validate_grid()

Extracting signatures for n_components = 9..................
n_components = 9, replicate 0 finished.
Time elapsed: 2.69 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.71 seconds.


...

/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.67 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


In [16]:
# Best parameter:
print(model.best_grid_point)
print(model.thresh_match)
print(model.thresh_refit)

(0.1, 0.0001)
0.1
0.0001


In [17]:
# Assigned signatures
print(model.sigs_assigned)

['SBS1' 'SBS2' 'SBS3' 'SBS5' 'SBS8' 'SBS10c' 'SBS13' 'SBS17a' 'SBS17b'
 'SBS18' 'SBS30' 'SBS39' 'SBS44' 'SBS98']


In [18]:
# Number of assigned signatures on the grid:
model.n_sigs_assigned_grid

{(0.001, 0.0001): 49,
 (0.01, 0.0001): 27,
 (0.1, 0.0001): 14,
 (1.0, 0.0001): 8,
 (5.0, 0.0001): 8}

In [19]:
# W distance on the grid:
# Note that the grid point (5, 0.0001) is skipped because it is redundant with (1, 0.0001). 
# So that grid point information will not be shown. 
model.W_cos_dist_mean_grid

{(0.001, 0.0001): 0.036868671166777695,
 (0.01, 0.0001): 0.04462682459208654,
 (0.1, 0.0001): 0.05654013296059736,
 (1.0, 0.0001): 0.19444173538214649}

In [20]:
# H distance on the grid:
model.H_frobenius_dist_mean_grid

{(0.001, 0.0001): 8623.372977043886,
 (0.01, 0.0001): 11678.137089219019,
 (0.1, 0.0001): 12320.102283658041,
 (1.0, 0.0001): 25373.23275033778}

In [21]:
# You can access the matrices as follows:
# model.W_simul_grid is a dictionary
# Each element of it is a list of matrices resulting from validate_n_replicates number of replicates. 
# Since by default validate_n_replicates = 1. It is simply a list of a single pd.DataFrame
model.W_simul_grid[(0.1, 0.0001)][0]

,Sig1,Sig2,Sig3,Sig4,Sig5,Sig6,Sig7,Sig8,Sig9
A[C>A]A,2.777548e-04,1.702278e-03,0.006478,6.511908e-03,0.013842,1.099392e-02,0.041619,3.472538e-02,0.014127
A[C>A]C,7.309300e-04,2.011518e-04,0.006693,7.011254e-03,0.003388,9.557090e-03,0.018682,3.818586e-02,0.016757
A[C>A]G,1.192093e-07,4.406919e-04,0.000919,1.988921e-04,0.001766,1.192093e-07,0.003285,4.600334e-03,0.002255
A[C>A]T,1.192093e-07,2.702002e-04,0.005243,1.540478e-03,0.011835,8.520790e-03,0.018315,4.359858e-02,0.008317
C[C>A]A,1.183345e-03,2.993167e-04,0.005810,9.323062e-03,0.000010,1.258654e-02,0.055703,2.038023e-02,0.022612
...,...,...,...,...,...,...,...,...,...
G[T>G]T,1.151192e-04,1.192093e-07,0.044014,1.192093e-07,0.000109,4.753002e-03,0.002237,1.192093e-07,0.006807
T[T>G]A,1.192093e-07,3.199905e-04,0.003428,3.144234e-03,0.008224,1.435524e-03,0.001465,1.192093e-07,0.009608
T[T>G]C,1.192093e-07,5.476069e-05,0.006316,3.766968e-03,0.009271,9.367320e-04,0.000900,8.330717e-04,0.006685
T[T>G]G,6.651044e-04,1.192093e-07,0.004809,3.202703e-03,0.007580,5.238418e-03,0.000845,1.053327e-03,0.010744


In [22]:
model.H_simul_grid[(0.1, 0.0001)][0]
# You can also get X_simul from: model.X_simul_grid

,Breast-AdenoCA::SP117956,Breast-AdenoCA::SP117975,Breast-AdenoCA::SP2149,Breast-AdenoCA::SP117113,Breast-AdenoCA::SP117245,Breast-AdenoCA::SP117402,Breast-AdenoCA::SP117136,Breast-AdenoCA::SP117378,Breast-AdenoCA::SP116947,Breast-AdenoCA::SP117369,...,Breast-AdenoCA::SP2731,Breast-AdenoCA::SP6223,Breast-AdenoCA::SP7171,Breast-AdenoCA::SP10563,Breast-AdenoCA::SP8987,Breast-AdenoCA::SP5980,Breast-AdenoCA::SP12186,Breast-AdenoCA::SP10944,Breast-AdenoCA::SP2766,Breast-AdenoCA::SP6673
Sig1,64.416695,100.893803,46.080525,106.481053,341.487598,599.305959,15.416401,47.579397,102.353988,27990.748814,...,418.871232,220.873716,510.483028,1402.468626,414.084811,56.684436,2437.531356,3040.790224,21.972086,3219.494334
Sig2,27.360836,123.048244,865.618429,0.000000,141.769973,1227.361762,33.521678,18.994616,143.903127,28690.327484,...,375.883614,874.536015,187.342516,526.968798,1750.990503,1493.654770,3731.288319,3398.039875,721.398443,5738.810853
Sig3,12.198447,17.449869,336.265697,22.271418,62.985527,35.937743,23.101537,22.908317,9.087464,12.127002,...,61.821163,265.325477,166.530316,64.967332,381.664485,286.447427,88.818229,127.853134,217.511479,781.518792
Sig4,389.856897,872.201576,530.870261,691.541509,696.408905,488.545315,504.766537,350.490224,741.143618,0.000000,...,455.313891,409.105321,632.046366,943.662750,354.764600,376.224952,1317.253880,1658.349793,206.912915,16.154600
Sig5,392.039406,205.867083,1498.447023,214.175990,357.494708,502.538397,187.095431,452.069991,369.700757,277.602699,...,252.202634,984.697136,583.029971,705.255052,1178.308343,1102.762045,476.224902,1221.286624,1226.073262,1433.087656
Sig6,553.635388,564.808748,197.484903,378.053463,262.998443,423.903508,393.012730,587.470905,628.647947,0.000000,...,495.133678,530.721003,579.405430,698.694594,753.929055,433.638044,67.359914,1486.831703,704.257206,1010.272840
Sig7,9.778483,116.594442,1062.969902,442.148464,263.089045,163.284265,188.386852,42.537368,64.297499,0.000000,...,517.302573,574.893332,472.553465,346.490414,1030.670539,791.385223,1213.194883,398.816525,712.084063,1624.833730
Sig8,352.561321,238.431044,2242.519135,246.464042,204.705563,183.907936,97.423748,88.164529,382.539590,0.000000,...,60.975054,2552.010740,308.191285,217.148533,1348.777125,2050.486182,207.522734,1090.278240,1790.087420,5287.844858
Sig9,114.259866,229.963778,4017.073294,0.000000,217.386930,383.644326,54.084694,197.769002,264.264153,0.000000,...,157.945523,1984.188786,355.112369,54.891203,3665.094676,3586.257746,0.000000,1189.453272,2173.186762,3850.480348


We can then do a 1d search for refitting

In [23]:
model.assign_grid(W_catalog, 
                  thresh_match_grid=np.array([0.1]), 
                  thresh_refit_grid=np.array([0.001, 0.01, 0.1, 1, 5]), 
                  thresh_new_sig=0.0)

/Users/hujin/GitHub/MuSiCal/musical/denovo.py:1056: UserWarning: self.assign_grid has been previously called. This call will replace the previous results.
  UserWarning)


In [24]:
model.validate_grid()

Extracting signatures for n_components = 9..................
n_components = 9, replicate 0 finished.
Time elapsed: 2.79 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.6 seconds.


...

/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)


Extracting signatures for n_components = 9..................


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


n_components = 9, replicate 0 finished.
Time elapsed: 2.7 seconds.


/Users/hujin/GitHub/MuSiCal/musical/cluster.py:181: RuntimeWarning: divide by zero encountered in log
  self.Wk_log = np.log(self.Wk)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:195: RuntimeWarning: divide by zero encountered in log
  self.Wk_log_ref_all = np.log(self.Wk_ref_all)
//anaconda3/envs/python37_musical/lib/python3.7/site-packages/numpy/core/_methods.py:229: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:206: RuntimeWarning: invalid value encountered in subtract
  self.gap_statistic_log = self.Wk_log_ref - self.Wk_log
/Users/hujin/GitHub/MuSiCal/musical/cluster.py:213: UserWarning: Based on the gap statistic, no k value is a valid candidate. The greatest k is chosen.
  UserWarning)
/Users/hujin/GitHub/MuSiCal/musical/denovo.py:360: UserWarning: Only 1 n_components value is tested. Selecting this n_components value.
  UserWarning)


In [25]:
# Best parameter:
print(model.best_grid_point)
print(model.thresh_match)
print(model.thresh_refit)

(0.1, 0.001)
0.1
0.001


In [26]:
# Assigned signatures
print(model.sigs_assigned)

['SBS1' 'SBS2' 'SBS3' 'SBS5' 'SBS8' 'SBS10c' 'SBS13' 'SBS17a' 'SBS17b'
 'SBS18' 'SBS30' 'SBS39' 'SBS44' 'SBS98']


In [27]:
# W distance on the grid:
model.W_cos_dist_mean_grid

{(0.1, 0.001): 0.05845970372025019,
 (0.1, 0.01): 0.08521209723889488,
 (0.1, 0.1): 0.0907986408751297,
 (0.1, 1.0): 0.3264700952507853,
 (0.1, 5.0): 0.3109373447523275}

In [28]:
# H distance on the grid:
model.H_frobenius_dist_mean_grid

{(0.1, 0.001): 12530.04633120366,
 (0.1, 0.01): 22154.218852077123,
 (0.1, 0.1): 38366.97194555916,
 (0.1, 1.0): 79102.59575351811,
 (0.1, 5.0): 68663.8745176915}

In [29]:
# Final result
model.W_s

,SBS1,SBS2,SBS3,SBS5,SBS8,SBS10c,SBS13,SBS17a,SBS17b,SBS18,SBS30,SBS39,SBS44,SBS98
Type,,,,,,,,,,,,,,
A[C>A]A,8.861572e-04,5.800168e-07,0.020808,0.011998,4.409822e-02,0.004331,1.819920e-03,2.070261e-03,0.000608,0.051534,0.001800,0.011702,7.681946e-18,0.013372
A[C>A]C,2.280405e-03,1.480043e-04,0.016507,0.009438,4.779807e-02,0.014830,7.209682e-04,9.181159e-04,0.000129,0.015810,0.000506,0.007151,1.500380e-04,0.010144
A[C>A]G,1.770314e-04,5.230151e-05,0.001751,0.001850,4.619813e-03,0.000657,2.639884e-04,4.760601e-05,0.000058,0.002432,0.000091,0.002670,9.162321e-07,0.002156
A[C>A]T,1.280227e-03,9.780282e-05,0.012205,0.006609,4.699810e-02,0.013128,3.479847e-04,6.180780e-05,0.000456,0.021414,0.000556,0.007401,5.781464e-03,0.012239
C[C>A]A,3.120554e-04,2.080060e-04,0.022509,0.007429,4.009838e-02,0.017433,1.399938e-03,2.950373e-04,0.000271,0.074049,0.000768,0.010101,2.840719e-05,0.007782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G[T>G]T,1.460259e-05,2.230064e-16,0.005832,0.002350,1.289948e-03,0.007474,2.889873e-06,7.240914e-03,0.113992,0.002141,0.000160,0.005941,2.800709e-03,0.004398
T[T>G]A,2.230396e-16,1.670048e-05,0.007253,0.005219,2.219910e-16,0.008731,1.839919e-04,2.220280e-16,0.000012,0.000696,0.000118,0.008391,7.681946e-18,0.008400
T[T>G]C,5.510978e-05,7.040203e-05,0.006283,0.006559,1.139954e-03,0.000952,2.219902e-16,1.160146e-04,0.008749,0.002101,0.000096,0.005081,7.681946e-18,0.005467


In [30]:
# Final result
model.H_s

,Breast-AdenoCA::SP117956,Breast-AdenoCA::SP117975,Breast-AdenoCA::SP2149,Breast-AdenoCA::SP117113,Breast-AdenoCA::SP117245,Breast-AdenoCA::SP117402,Breast-AdenoCA::SP117136,Breast-AdenoCA::SP117378,Breast-AdenoCA::SP116947,Breast-AdenoCA::SP117369,...,Breast-AdenoCA::SP2731,Breast-AdenoCA::SP6223,Breast-AdenoCA::SP7171,Breast-AdenoCA::SP10563,Breast-AdenoCA::SP8987,Breast-AdenoCA::SP5980,Breast-AdenoCA::SP12186,Breast-AdenoCA::SP10944,Breast-AdenoCA::SP2766,Breast-AdenoCA::SP6673
SBS1,227.601503,457.160361,404.280282,367.371188,371.337474,331.300048,252.010274,198.480341,377.078760,106.117178,...,258.318492,375.777692,323.104704,570.912510,414.783905,331.870440,680.748138,1060.529132,301.357738,341.594623
SBS2,0.000000,107.053668,140.680529,69.741021,283.747970,783.492585,31.846222,45.726073,101.368798,30482.225452,...,394.964010,281.496103,445.383040,1333.567034,671.404221,332.486724,2847.731092,3302.634724,118.545450,3769.113236
SBS3,0.000000,299.883872,2135.104747,0.000000,0.000000,601.424700,0.000000,0.000000,0.000000,0.000000,...,0.000000,1477.392605,0.000000,0.000000,4210.073754,2600.410499,0.000000,1808.499374,2269.145098,4035.330269
SBS5,828.826492,792.152608,3086.422701,486.976318,903.509102,565.290798,771.623358,1305.995853,1347.770131,0.000000,...,451.979145,1495.147090,1195.261570,1224.188335,1750.177161,2062.561847,1022.115168,2176.907573,1959.279980,2337.319731
SBS8,356.968656,138.951752,2659.913665,114.493946,272.646212,109.109287,180.337041,110.074358,425.742288,0.000000,...,179.268072,2210.625648,377.599818,245.644423,1040.506286,2007.994010,366.342840,863.796646,1532.020449,4889.186485
SBS10c,0.000000,0.000000,0.000000,131.125114,76.218321,0.000000,62.807327,0.000000,0.000000,0.000000,...,197.848782,0.000000,0.000000,0.000000,542.787124,0.000000,212.287920,342.928531,256.582547,0.000000
SBS13,30.129458,71.801033,294.026473,0.000000,160.094613,928.973605,0.000000,0.000000,99.885638,24061.911647,...,276.978155,547.644384,166.487556,477.034237,1092.887839,813.576369,3107.907264,2796.050177,348.492265,4403.434051
SBS17a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SBS17b,0.000000,0.000000,81.756267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,60.095924,39.436578,0.000000,0.000000,73.966222,0.000000,0.000000,0.000000,229.163422
SBS18,0.000000,143.663640,0.000000,364.515423,90.270607,145.390987,88.878319,41.481843,0.000000,0.000000,...,211.507393,0.000000,359.870810,284.253559,0.000000,0.000000,694.039421,0.000000,0.000000,0.000000
